In [7]:
suppressMessages(library(ggplot2))
suppressMessages(library(ArchR))
suppressMessages(library(patchwork))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(ggrepel))
suppressMessages(library(rhdf5))
suppressMessages(library(chromVARmotifs))

In [8]:
set.seed(42)
addArchRThreads(threads = 64)
addArchRGenome("hg38")

Setting default number of Parallel threads to 64.

Setting default genome to Hg38.



In [9]:
proj <- loadArchRProject("./ArchRProject", showLogo = FALSE)

Successfully loaded ArchRProject!



In [6]:
length(proj$cellNames)

[1] 86017

In [5]:
## 计算覆盖度
### 覆盖度：计算每个样本中的reads对基因组的覆盖度，表示每个细胞的测序深度。
### maxCells和minCells，限制每个样本中用于计算覆盖度的细胞（超过5000个时，随机选择5000个来计算覆盖度）
proj <- addGroupCoverages(ArchRProj = proj, groupBy = "Sample",
                          maxCells = 5000,
                          minCells = 5000)

ArchR logging to : ArchRLogs/ArchR-addGroupCoverages-c314c33301f16-Date-2025-02-24_Time-12-26-52.408171.log
If there is an issue, please report to github with logFile!



10N (1 of 11) : CellGroups N = 2

10T (2 of 11) : CellGroups N = 2

11T (3 of 11) : CellGroups N = 2

12N (4 of 11) : CellGroups N = 2

12T (5 of 11) : CellGroups N = 2

13T (6 of 11) : CellGroups N = 2

18T (7 of 11) : CellGroups N = 2

19T (8 of 11) : CellGroups N = 2

20T (9 of 11) : CellGroups N = 2

21T (10 of 11) : CellGroups N = 2

22T (11 of 11) : CellGroups N = 2

2025-02-24 12:26:57.540692 : Further Sampled 22 Groups above the Max Fragments!, 0.086 mins elapsed.

2025-02-24 12:26:58.094321 : Creating Coverage Files!, 0.095 mins elapsed.

2025-02-24 12:26:58.097149 : Batch Execution w/ safelapply!, 0.095 mins elapsed.

2025-02-24 12:26:58.150272 : Group 10N._.Rep1 (1 of 22) : Creating Group Coverage File : X10N._.Rep1.insertions.coverage.h5, 0.096 mins elapsed.

Number of Cells = 2545

Coverage File Exists!

Added Coverage Group

Added Metadata Group

Added ArrowCoverage Class

Added Coverage/Info

Added Coverage/Info/CellNames

2025-02-24 12:27:52.585872 : Group 10N._.Rep2 (2

In [10]:
?addReproduciblePeakSet

addReproduciblePeakSet          package:ArchR          R Documentation

_A_d_d _a _R_e_p_r_o_d_u_c_i_b_l_e _P_e_a_k _S_e_t _t_o _a_n _A_r_c_h_R_P_r_o_j_e_c_t

_D_e_s_c_r_i_p_t_i_o_n:

     This function will get insertions from coverage files, call peaks,
     and merge peaks to get a "Union Reproducible Peak Set".

_U_s_a_g_e:

     addReproduciblePeakSet(
       ArchRProj = NULL,
       groupBy = "Clusters",
       peakMethod = "Macs2",
       reproducibility = "2",
       peaksPerCell = 500,
       maxPeaks = 150000,
       minCells = 25,
       excludeChr = c("chrM", "chrY"),
       pathToMacs2 = if (tolower(peakMethod) == "macs2") findMacs2() else NULL,
       genomeSize = NULL,
       shift = -75,
       extsize = 150,
       method = if (tolower(peakMethod) == "macs2") "q" else "p",
       cutOff = 0.1,
       additionalParams = "--nomodel --nolambda",
       extendSummits = 250,
       promoterRegion = c(2000, 100),
       

In [ ]:
## peak calling
## 使用macs2进行peak calling
pathToMacs2 <- findMacs2()

## 添加一个可重复的峰值集
## 对每个样本进行峰值调用，然后将峰值集合并，过滤掉不可重复的峰值（cutoff）
## cutoff，峰值的p值阈值，用于过滤峰值
proj <- addReproduciblePeakSet(
    ArchRProj = proj, 
    groupBy = "Sample", 
    cutOff = 0.05,
    minCells = 25,
    promoterRegion = c(2000, 100),
    pathToMacs2 = pathToMacs2
)

proj <- addPeakMatrix(proj)

Searching For MACS2..

Found with $PATH at /home/hanxue/miniconda3/bin/macs2

ArchR logging to : ArchRLogs/ArchR-addReproduciblePeakSet-c314c659ab309-Date-2025-02-24_Time-13-16-08.279945.log
If there is an issue, please report to github with logFile!

Calling Peaks with Macs2

2025-02-24 13:16:08.421258 : Peak Calling Parameters!, 0.002 mins elapsed.



    Group nCells nCellsUsed nReplicates nMin nMax maxPeaks
10N   10N  10296      10000           2 5000 5000   150000
10T   10T  10218      10000           2 5000 5000   150000
11T   11T   8644       7178           2 5000 5000   150000
12N   12N   7346       6655           2 5000 5000   150000
12T   12T   7089       6516           2 5000 5000   150000
13T   13T   9695       7500           2 5000 5000   150000
18T   18T   6068       4935           2 3393 3397   150000
19T   19T   6367       6103           2 5000 5000   150000
20T   20T   7036       6486           2 5000 5000   150000
21T   21T   7774       6832           2 5000 5000   150000
22T   22T   5484       4637           2 3256 3272   150000


2025-02-24 13:16:08.43061 : Batching Peak Calls!, 0.003 mins elapsed.

2025-02-24 13:16:08.438231 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-02-24 13:19:58.590448 : Identifying Reproducible Peaks!, 3.838 mins elapsed.



rno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call


2025-02-24 13:20:08.031703 : Creating Union Peak Set!, 3.996 mins elapsed.

Converged after 9 iterations!

Plotting Ggplot!

2025-02-24 13:20:19.905683 : Finished Creating Union Peak Set (345310)!, 4.194 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-c314c47033639-Date-2025-02-24_Time-13-20-19.909908.log
If there is an issue, please report to github with logFile!

2025-02-24 13:20:20.01029 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addPeakMatrix-c314c47033639-Date-2025-02-24_Time-13-20-19.909908.log



In [12]:
## save peak matrix
## 峰值矩阵peakMatrix中的计数矩阵counts
## df_rangers，峰值的坐标信息
peakMatrix <- getMatrixFromProject(proj,
                                   useMatrix = "PeakMatrix")

counts <- peakMatrix@assays@data$PeakMatrix
df_rangers <- as.data.frame(peakMatrix@rowRanges@ranges)

## 格式化counts的行名为染色体_起始位置_终止位置
rownames(counts) <- paste(peakMatrix@rowRanges@seqnames,
                          df_rangers$start,
                          df_rangers$end,
                          sep = "_") 
# 保存峰值矩阵
saveRDS(counts, file = "./ArchRProject/PeakMatrix.Rds")

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-c314c63da3125-Date-2025-02-24_Time-13-28-45.656913.log
If there is an issue, please report to github with logFile!



2025-02-24 13:29:57.77394 : Organizing colData, 1.202 mins elapsed.

2025-02-24 13:29:58.114991 : Organizing rowData, 1.208 mins elapsed.

2025-02-24 13:29:58.131417 : Organizing rowRanges, 1.208 mins elapsed.

2025-02-24 13:29:58.153352 : Organizing Assays (1 of 1), 1.208 mins elapsed.

2025-02-24 13:30:40.266477 : Constructing SummarizedExperiment, 1.91 mins elapsed.

2025-02-24 13:31:50.698181 : Finished Matrix Creation, 3.084 mins elapsed.



In [13]:
## save gene matrix
## 和峰值矩阵类似，基因矩阵GeneScoreMatrix中的计数矩阵counts，保存基因矩阵
atac <- getMatrixFromProject(ArchRProj = proj,
                             useMatrix = "GeneScoreMatrix")

counts <- atac@assays@data$GeneScoreMatrix
rownames(counts) <- atac@elementMetadata$name

saveRDS(counts, file = "./ArchRProject/GeneScoreMatrix.Rds")

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-c314c3eb91604-Date-2025-02-24_Time-13-47-17.04584.log
If there is an issue, please report to github with logFile!

2025-02-24 13:48:18.62827 : Organizing colData, 1.026 mins elapsed.

2025-02-24 13:48:19.0466 : Organizing rowData, 1.033 mins elapsed.

2025-02-24 13:48:19.054247 : Organizing rowRanges, 1.033 mins elapsed.

2025-02-24 13:48:19.061831 : Organizing Assays (1 of 1), 1.034 mins elapsed.

2025-02-24 13:49:08.418415 : Constructing SummarizedExperiment, 1.856 mins elapsed.

2025-02-24 13:49:10.354281 : Finished Matrix Creation, 1.888 mins elapsed.



In [14]:
## 添加motif注释
proj <- addMotifAnnotations(proj, motifSet = "homer", annoName = "homer")
# 添加背景峰值，添加峰值偏差矩阵
## 峰值偏差矩阵
proj <- addBgdPeaks(proj) %>% addDeviationsMatrix(., peakAnnotation = "homer")
# 检查当前ArchRProject中存放了哪些矩阵数据
getAvailableMatrices(proj)

ArchR logging to : ArchRLogs/ArchR-addMotifAnnotations-c314c56366810-Date-2025-02-24_Time-14-18-33.111154.log
If there is an issue, please report to github with logFile!

2025-02-24 14:18:33.712746 : Getting Motif Set, Species : , 0.002 mins elapsed.

2025-02-24 14:18:35.92506 : Finding Motif Positions with motifmatchr!, 0.039 mins elapsed.

2025-02-24 14:23:28.990012 : All Motifs Overlap at least 1 peak!, 4.923 mins elapsed.

2025-02-24 14:23:29.024291 : Creating Motif Overlap Matrix, 4.924 mins elapsed.

2025-02-24 14:23:33.343299 : Finished Getting Motif Info!, 4.996 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addMotifAnnotations-c314c56366810-Date-2025-02-24_Time-14-18-33.111154.log

Identifying Background Peaks!

Using Previous Background Peaks!

ArchR logging to : ArchRLogs/ArchR-addDeviationsMatrix-c314c35b6cf0d-Date-2025-02-24_Time-14-24-56.204333.log
If there is an issue, please report to github with logFile!

2025-02-24 14:25:14.170182 : Batch Execution w/ sa

[1] "GeneScoreMatrix" "PeakMatrix"      "TileMatrix"      "homerMatrix"

In [15]:
saveArchRProject(ArchRProj = proj, 
                 load = FALSE)
sessionInfo()

Saving ArchRProject...



R version 4.4.1 (2024-06-14)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /home/hanxue/miniconda3/envs/scseq/lib/libopenblasp-r0.3.21.so;  LAPACK version 3.9.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] chromVAR_1.26.0                   motifmatchr_1.26.0               
 [3] chromVARmotifs_0.2.0              ggrepel_0.9.6                    
 [5] cowplot_1.1.3    